# Imports

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Data Bases

In [6]:
#Import the training data
train = pd.read_csv('https://github.com/tcastrom/CEFR-French-/raw/main/Data/Data%20with%20Readability%20Metrics/train_metrics.csv')
#rename colun Unnamed: 0	to id
train.rename(columns={'Unnamed: 0':'id'}, inplace=True)
train.set_index('id', inplace=True)
display(train.head())

#Import the unlabel data
unlabel = pd.read_csv('https://github.com/tcastrom/CEFR-French-/raw/main/Data/Data%20with%20Readability%20Metrics/unlabel_metrics.csv')
unlabel.set_index('id', inplace=True)
display(unlabel.head())

,sentence,difficulty,flesch_reading_ease,polysyllable_count,coleman_liau_index,freqlemfilms2_mean,freqlemfilms2_median,freqlemfilms2_max,freqlemfilms2_min,freqlemfilms2_25%,...,nblettres_max,nblettres_min,nblettres_25%,nblettres_75%,nbsyll_mean,nbsyll_median,nbsyll_max,nbsyll_min,nbsyll_25%,nbsyll_75%
id,,,,,,,,,,,,,,,,,,,,,
0,Les coûts kilométriques réels peuvent diverger...,C1,43.31,9,15.39,5083.383947,86.650,25220.86,0.04,13.9300,...,13.0,1.0,2.0,7.0,1.763158,1.0,4.0,1.0,1.0,2.0
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,99.14,1,4.57,6604.464167,1214.575,25983.20,8.21,46.2975,...,8.0,1.0,2.0,4.0,1.250000,1.0,3.0,1.0,1.0,1.0
2,Le test de niveau en français est sur le site ...,A1,90.77,1,5.03,9111.054615,2520.110,32236.50,5.61,50.7000,...,8.0,1.0,2.0,4.0,1.307692,1.0,3.0,1.0,1.0,1.0
3,Est-ce que ton mari est aussi de Boston?,A1,95.84,0,2.86,11378.233333,4100.900,32236.50,0.44,1402.3300,...,6.0,2.0,3.0,4.0,1.333333,1.0,2.0,1.0,1.0,2.0
4,"Dans les écoles de commerce, dans les couloirs...",B1,69.45,2,11.79,6372.454516,1252.420,25220.86,2.02,54.6800,...,11.0,1.0,2.0,6.0,1.354839,1.0,3.0,1.0,1.0,2.0


,sentence,flesch_reading_ease,polysyllable_count,coleman_liau_index,freqlemfilms2_mean,freqlemfilms2_median,freqlemfilms2_max,freqlemfilms2_min,freqlemfilms2_25%,freqlemfilms2_75%,...,nblettres_max,nblettres_min,nblettres_25%,nblettres_75%,nbsyll_mean,nbsyll_median,nbsyll_max,nbsyll_min,nbsyll_25%,nbsyll_75%
id,,,,,,,,,,,,,,,,,,,,,
0,Nous dûmes nous excuser des propos que nous eû...,86.45,2,10.24,4181.733000,4436.510,13572.40,24.99,1107.5350,4772.120,...,9.0,3.0,4.00,5.75,1.500000,1.0,3.0,1.0,1.0,1.75
1,Vous ne pouvez pas savoir le plaisir que j'ai ...,89.75,1,8.63,7940.903571,4308.805,25220.86,64.31,533.1575,13636.995,...,8.0,1.0,2.25,6.00,1.428571,1.0,3.0,1.0,1.0,2.00
2,"Et, paradoxalement, boire froid n'est pas la b...",80.11,1,10.58,5342.156667,339.050,18188.15,0.22,9.6700,12909.080,...,14.0,1.0,2.00,5.00,1.666667,1.0,6.0,1.0,1.0,1.00
3,"Ce n'est pas étonnant, car c'est une saison my...",94.83,1,9.31,3515.732222,31.950,18188.15,8.21,19.6100,5219.100,...,11.0,1.0,2.00,6.00,1.555556,1.0,3.0,1.0,1.0,2.00
4,"Le corps de Golo lui-même, d'une essence aussi...",8.80,12,14.30,5236.215139,1174.325,32236.50,0.01,38.3100,5559.810,...,14.0,1.0,2.00,6.25,1.555556,1.0,5.0,1.0,1.0,2.00


# Encoding of the Data

In [7]:
# Encode the difficulty column using LabelEncoder
le = LabelEncoder()
train['difficulty'] = le.fit_transform(train['difficulty'])

# Display how the difficulty has been encoded and the values associated with each encoding
display(le.classes_)
print(f'The values [0, 1, 2, 3, 4, 5] are represented by{le.inverse_transform([0, 1, 2, 3, 4, 5])}')

array(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], dtype=object)

The values [0, 1, 2, 3, 4, 5] are represented by['A1' 'A2' 'B1' 'B2' 'C1' 'C2']


In [8]:
# Create X and y
X = train.drop(columns=['difficulty'])
y = train['difficulty']

# Split the data into train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Assuming X_train['sentence'] is the column containing the sentences
vectorizer = TfidfVectorizer() 

# Fit and transform the sentences in the training set
X_train_matrix = vectorizer.fit_transform(X_train['sentence'])

# Transform the sentences in the test set
X_test_matrix = vectorizer.transform(X_test['sentence'])

# Convert the sparse matrix to a DataFrame
X_train_vectorized = pd.DataFrame(X_train_matrix.toarray(), columns=vectorizer.get_feature_names_out())
X_test_vectorized = pd.DataFrame(X_test_matrix.toarray(), columns=vectorizer.get_feature_names_out())


In [10]:
# Drop the 'sentence' column from the training and testing sets
X_train_non_text = X_train.drop(columns=['sentence']).reset_index(drop=True)
X_test_non_text = X_test.drop(columns=['sentence']).reset_index(drop=True)

# Scale the non-text features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_non_text), columns=X_train_non_text.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_non_text), columns=X_test_non_text.columns)

# Combine the scaled non-text features with the text features
X_train = pd.concat([X_train_scaled, X_train_vectorized.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test_scaled, X_test_vectorized.reset_index(drop=True)], axis=1)

# Display the updated DataFrame
display(X_train.head())
display(X_test.head())

,flesch_reading_ease,polysyllable_count,coleman_liau_index,freqlemfilms2_mean,freqlemfilms2_median,freqlemfilms2_max,freqlemfilms2_min,freqlemfilms2_25%,freqlemfilms2_75%,freqlemlivres_mean,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,1.700307,-0.836635,-1.903433,0.497089,0.513624,-0.690838,-0.161413,-0.324706,0.904663,-0.321257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.393045,-0.836635,-1.852676,1.620494,2.292458,0.196495,2.689333,0.532810,1.267484,2.287844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.426575,-0.172790,0.417849,1.759942,-0.449512,1.130980,0.100366,-0.271439,2.322918,1.483208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.402858,0.491055,0.803602,-1.111185,-0.724094,0.196495,-0.193968,-0.360484,-1.034622,-0.764438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.700307,-0.836635,-1.243593,2.319136,4.097446,0.298039,0.578606,0.874354,2.066949,0.118607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,flesch_reading_ease,polysyllable_count,coleman_liau_index,freqlemfilms2_mean,freqlemfilms2_median,freqlemfilms2_max,freqlemfilms2_min,freqlemfilms2_25%,freqlemfilms2_75%,freqlemlivres_mean,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,1.076617,-0.836635,-1.375561,1.143169,0.618569,0.298039,0.075155,1.336927,0.404896,-1.167305,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.630024,-0.836635,0.008410,-0.268043,-0.507446,0.196495,-0.178731,-0.367098,-0.017557,0.849703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.272694,1.154901,1.048927,-0.685076,-0.740714,0.196495,-0.192515,-0.374812,-0.722947,-0.259846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.285730,-0.172790,0.145454,-0.570586,-0.738913,-0.194186,-0.137890,-0.348110,-0.483589,-0.856393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.546426,-0.504713,-0.996576,0.584402,-0.638657,0.196495,-0.179752,-0.267138,0.907415,1.709151,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Linear Regression

In [11]:
# Do a linear regression 
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Predict the difficulty on the testing data
y_pred = linear_regression.predict(X_test)

# Round the predictions to the nearest integer and make sure the values are between 0 and 5
y_pred = np.round(y_pred)
y_pred[y_pred < 0] = 0
y_pred[y_pred > 5] = 5

# Reverse the encoding of the difficulty
y_test_unencoded = le.inverse_transform(y_test)
y_pred_unencoded = le.inverse_transform(y_pred.astype(int))

# Calculate the accuracy
accuracy_reg_lin = accuracy_score(y_test_unencoded, y_pred_unencoded)
print(f'Accuracy: {accuracy_reg_lin}')

# Calculate the Precision 
precision_reg_lin = precision_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Precision: {precision_reg_lin}')

# Calculate the Recall
recall_reg_lin = recall_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Recall: {recall_reg_lin}')

# Calculate the F1 Score
f1_reg_lin = f1_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'F1 Score: {f1_reg_lin}')

Accuracy: 0.175
Precision: 0.06025681362776226
Recall: 0.175
F1 Score: 0.08963637107591736


c:\Users\castr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic  Regression with cross validation

In [12]:
# Do a logistic regression woth cross validation
from sklearn.linear_model import LogisticRegressionCV

# Create a logistic regression object
logistic_regression = LogisticRegressionCV(cv=5, max_iter=1000)

# Fit the model on the training data
logistic_regression.fit(X_train, y_train)

# Predict the difficulty on the testing data
y_pred = logistic_regression.predict(X_test)

# Reverse the encoding of the difficulty
y_test_unencoded = le.inverse_transform(y_test)
y_pred_unencoded = le.inverse_transform(y_pred)

# Calculate the accuracy
accuracy_log_reg = accuracy_score(y_test_unencoded, y_pred_unencoded)
print(f'Accuracy: {accuracy_log_reg}')

# Calculate the Precision
precision_log_reg = precision_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Precision: {precision_log_reg}')

# Calculate the Recall
recall_log_reg = recall_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Recall: {recall_log_reg}')

# Calculate the F1 Score
f1_log_reg = f1_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'F1 Score: {f1_log_reg}')




c:\Users\castr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\castr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracy: 0.49895833333333334
Precision: 0.49479500760712836
Recall: 0.49895833333333334
F1 Score: 0.4957320510109303


# KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Define the parameter grid
param_grid_knn = {
    'n_neighbors': range(1, 11),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Create a base model
knn = KNeighborsClassifier()

# Instantiate the grid search model
grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid_knn, cv=5)

# Fit the grid search to the data
grid_search_knn.fit(X_train, y_train)

# Get the best parameters
best_params_knn = grid_search_knn.best_params_

print(f"The best parameters are : {best_params_knn}")

c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKm

The best parameters are : {'metric': 'manhattan', 'n_neighbors': 10, 'weights': 'distance'}


In [14]:
# Retrieve the best model
best_model_knn = grid_search_knn.best_estimator_

# Predict the difficulty on the testing data
y_pred = best_model_knn.predict(X_test)

# Reverse the encoding of the difficulty
y_test_unencoded = le.inverse_transform(y_test)
y_pred_unencoded = le.inverse_transform(y_pred)

# Calculate the accuracy
accuracy_knn = accuracy_score(y_test_unencoded, y_pred_unencoded)
print(f'Accuracy: {accuracy_knn}')

# Calculate the Precision
precision_knn = precision_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Precision: {precision_knn}')

# Calculate the Recall
recall_knn = recall_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Recall: {recall_knn}')

# Calculate the F1 Score
f1_knn = f1_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'F1 Score: {f1_knn}')

Accuracy: 0.37083333333333335
Precision: 0.3679626180650345
Recall: 0.37083333333333335
F1 Score: 0.3665112246603523


In [15]:
# Decision Tree with GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define the parameter grid
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(1, 11),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30]
}

# Create a base model
dt = DecisionTreeClassifier()

# Instantiate the grid search model
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=5)

# Fit the grid search to the data
grid_search_dt.fit(X_train, y_train)

# Get the best parameters
best_params_dt = grid_search_dt.best_params_

print(f"The best parameters are : {best_params_dt}")

c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
9600 fits failed out of a total of 28800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9600 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\castr\anaconda3\Lib\site-pa

The best parameters are : {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'max_leaf_nodes': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}


In [16]:
# Retrieve the best model
best_model_dt = grid_search_dt.best_estimator_

# Predict the difficulty on the testing data
y_pred = best_model_dt.predict(X_test)

# Reverse the encoding of the difficulty
y_test_unencoded = le.inverse_transform(y_test)
y_pred_unencoded = le.inverse_transform(y_pred)

# Calculate the accuracy
accuracy_dt = accuracy_score(y_test_unencoded, y_pred_unencoded)
print(f'Accuracy: {accuracy_dt}')

# Calculate the Precision
precision_dt = precision_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Precision: {precision_dt}')

# Calculate the Recall
recall_dt = recall_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Recall: {recall_dt}')

# Calculate the F1 Score
f1_dt = f1_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'F1 Score: {f1_dt}')


Accuracy: 0.29270833333333335
Precision: 0.277782620220236
Recall: 0.29270833333333335
F1 Score: 0.22864546533863836


# Random forest

In [17]:
# Random Forest with GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 11),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create a base model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5)

# Fit the grid search to the data
grid_search_rf.fit(X_train, y_train)

# Get the best parameters
best_params_rf = grid_search_rf.best_params_
print(f"The best parameters are : {best_params_rf}")

c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
3600 fits failed out of a total of 10800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3600 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\castr\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\castr\anaconda3\Lib\site-pa

The best parameters are : {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


In [18]:
# Retrieve the best model
best_model_rf = grid_search_rf.best_estimator_

# Predict the difficulty on the testing data
y_pred = best_model_rf.predict(X_test)

# Reverse the encoding of the difficulty
y_test_unencoded = le.inverse_transform(y_test)
y_pred_unencoded = le.inverse_transform(y_pred)

# Calculate the accuracy
accuracy_rf = accuracy_score(y_test_unencoded, y_pred_unencoded)
print(f'Accuracy: {accuracy_rf}')

# Calculate the Precision
precision_rf = precision_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Precision: {precision_rf}')

# Calculate the Recall
recall_rf = recall_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'Recall: {recall_rf}')

# Calculate the F1 Score
f1_rf = f1_score(y_test_unencoded, y_pred_unencoded, average='weighted')
print(f'F1 Score: {f1_rf}')

Accuracy: 0.44895833333333335
Precision: 0.44396565904840063
Recall: 0.44895833333333335
F1 Score: 0.43350532584952284


# Final Table

In [22]:
# Compile all the results into a Table. The columns are : Model, Accuracy, Precision, Recall, F1 Score 
from IPython.display import display

results = pd.DataFrame({
    'Model': ['Linear Regression', 'Logistic Regression', 'K-Nearest Neighbors', 'Decision Tree', 'Random Forest'],
    'Accuracy': [accuracy_reg_lin, accuracy_log_reg, accuracy_knn, accuracy_dt, accuracy_rf],
    'Precision': [precision_reg_lin, precision_log_reg, precision_knn, precision_dt, precision_rf],
    'Recall': [recall_reg_lin, recall_log_reg, recall_knn, recall_dt, recall_rf],
    'F1 Score': [f1_reg_lin, f1_log_reg, f1_knn, f1_dt, f1_rf]
})

display(results)

#Round the values to 2 decimal places
results = results.round(2)

display(results)

# Function to format numbers to 2 significant figures
def format_sigfig(x):
    if isinstance(x, float):
        return '{:.2g}'.format(x)
    return x

# Apply formatting function to the DataFrame
formatted_results = results.applymap(format_sigfig)


# Use the .style attribute for better formatting
styled_results = formatted_results.style.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', 'grey'), ('color', 'black'), ('font-weight', 'bold')]},
    {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f2f2f2'),('color', 'black'), ('font-weight', 'bold')]},
    {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', 'white'),('color', 'black'), ('font-weight', 'bold')]}
]).set_properties(**{'text-align': 'center'}).set_caption("Model Performance Comparison")

display(styled_results)

,Model,Accuracy,Precision,Recall,F1 Score
0,Linear Regression,0.175000,0.060257,0.175000,0.089636
1,Logistic Regression,0.498958,0.494795,0.498958,0.495732
2,K-Nearest Neighbors,0.370833,0.367963,0.370833,0.366511
3,Decision Tree,0.292708,0.277783,0.292708,0.228645
4,Random Forest,0.448958,0.443966,0.448958,0.433505


,Model,Accuracy,Precision,Recall,F1 Score
0,Linear Regression,0.18,0.06,0.18,0.09
1,Logistic Regression,0.50,0.49,0.50,0.50
2,K-Nearest Neighbors,0.37,0.37,0.37,0.37
3,Decision Tree,0.29,0.28,0.29,0.23
4,Random Forest,0.45,0.44,0.45,0.43


C:\Users\castr\AppData\Local\Temp\ipykernel_8168\397520918.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_results = results.applymap(format_sigfig)


,Model,Accuracy,Precision,Recall,F1 Score
0,Linear Regression,0.18,0.06,0.18,0.09
1,Logistic Regression,0.5,0.49,0.5,0.5
2,K-Nearest Neighbors,0.37,0.37,0.37,0.37
3,Decision Tree,0.29,0.28,0.29,0.23
4,Random Forest,0.45,0.44,0.45,0.43


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Model</th>
      <th>Accuracy</th>
      <th>Precision</th>
      <th>Recall</th>
      <th>F1 Score</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Linear Regression</td>
      <td>0.18</td>
      <td>0.06</td>
      <td>0.18</td>
      <td>0.09</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Logistic Regression</td>
      <td>0.50</td>
      <td>0.49</td>
      <td>0.50</td>
      <td>0.50</td>
    </tr>
    <tr>
      <th>2</th>
      <td>K-Nearest Neighbors</td>
      <td>0.37</td>
      <td>0.37</td>
      <td>0.37</td>
      <td>0.37</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Decision Tree</td>
      <td>0.29</td>
      <td>0.28</td>
      <td>0.29</td>
      <td>0.23</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Random Forest</td>
      <td>0.45</td>
      <td>0.44</td>
      <td>0.45</td>
      <td>0.43</td>
    </tr>
  </tbody>
</table>
</div>